# Filtros lineales espaciales - caso pasaalto

<div class="alert alert-block alert-success">
<b>Resumen:</b> En este notebook exploraremos los filtros pasa altos en el dominio espacial, específicamente utilizando el filtro Laplaciano. Aplicaremos estos filtros para resaltar bordes en una imagen y realizaremos una actividad de sharpening para mejorar la nitidez de la imagen original.
</div>

***

In [ ]:
# Importar las bibliotecas necesarias
import cv2
import numpy as np
import matplotlib.pyplot as plt

## 1. Lectura de la imagen

In [ ]:
# Leer la imagen en escala de grises
# Asegúrate de que 'test_pattern_blurring_orig.tif' esté en el mismo directorio que el notebook
im_gray_1 = cv2.imread('./images/test_pattern_blurring_orig.tif', cv2.IMREAD_GRAYSCALE)

# Verificar si la imagen se ha leído correctamente
if im_gray_1 is None:
    raise FileNotFoundError("La imagen 'test_pattern_blurring_orig.tif' no se encontró en el directorio actual.")

# Mostrar la imagen original
plt.figure(figsize=(5, 5))
plt.imshow(im_gray_1, cmap='gray')
plt.title('Imagen Original')
plt.axis('off')
plt.show()

### 2. Filtro pasaaltas con un Laplacian kernel utilizando OpenCV

El filtro Laplaciano es un operador que calcula la aproximación a la segunda derivada de la imagen, resaltando áreas donde la intensidad cambia rápidamente, es decir, los bordes.

### 2.1. Valores de escalado

In [ ]:
# Definir diferentes valores de escala
# En OpenCV, el parámetro 'scale' puede utilizarse para ajustar la intensidad del filtro
scales = [0.2, 0.2, 0.2]
mask_size = [3, 5, 7]

### 2.2. Filtrado

In [ ]:
# Aplicar el filtro Laplaciano con diferentes escalas
laplacian_results = []
for i,scale in enumerate(scales):
    # cv2.CV_32F permite obtener resultados en punto flotante para evitar saturación
    laplacian = cv2.Laplacian(im_gray_1, cv2.CV_32F, ksize=mask_size[i], scale=scale, borderType=cv2.BORDER_REPLICATE)
    laplacian_results.append(laplacian)

### 2.3. Visualización de resultados

In [ ]:
# Mostrar los resultados de los filtros Laplacianos
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for i, laplacian in enumerate(laplacian_results):
    # Convertir a formato adecuado para visualización
    laplacian_display = cv2.convertScaleAbs(laplacian)
    axes[i].imshow(laplacian_display, cmap='gray')
    axes[i].set_title(f'Filtro Laplaciano (scale={scale} y size={mask_size[i]})')
    axes[i].axis('off')

plt.show()

---

## 3. Resaltado de bordes

El objetivo del filtro de `sharpening` es ajustar la fusión entre la imagen original y el resultado del filtro Laplaciano para resaltar los bordes y mejorar la nitidez de la imagen.

### 3.1. Aplicar sharpening restando el Filtro Laplaciano

Para realizar el sharpening, combinamos la imagen original con el filtro Laplaciano. Existen varias formas de hacerlo, pero una común es restar el Laplaciano de la imagen original.

In [ ]:
# Convertir la imagen original a float para operaciones precisas
im_gray_1_float = im_gray_1.astype(np.float32)
alpha = 0.95

# Aplicar sharpening restando el filtro Laplaciano
sharpened_images = []
for laplacian in laplacian_results:
    sharpened = alpha*im_gray_1_float - (1-alpha)*laplacian

    # Normalizar la imagen para visualizar correctamente
    sharpened_norm = cv2.convertScaleAbs(sharpened)
    sharpened_images.append(sharpened_norm.astype(np.uint8))

### 3.2. Resultados

In [ ]:
# Mostrar los resultados de sharpening
fig, axes = plt.subplots(1, 4, figsize=(18, 6))

for i, scale in enumerate(scales):
    axes[i].imshow(sharpened_images[i], cmap='gray')
    axes[i].set_title(f'Sharpening (scale={scale})')
    axes[i].axis('off')

axes[3].imshow(im_gray_1, cmap='gray')
axes[3].set_title(f'Original')
axes[3].axis('off')

plt.show()